# Pretrain Model

In [1]:
# Import necessary libraries
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW

# Define the MathDataset class
class MathDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        problem = self.dataframe.iloc[idx]["problem"]
        solution = self.dataframe.iloc[idx]["solution"]

        return problem, solution

# collate_fn function to handle padding and tokenization for a whole batch
def collate_fn(batch):
    problems, solutions = zip(*batch)
    split_token = " =" # has additional space in front as this is a special token
    split_token_id = tokenizer.encode(split_token)[0]

    questions = [f"{p} {s}{tokenizer.eos_token}" for p, s in zip(problems, solutions)] # concatenate and add eos_token

    encoder = tokenizer(
        questions,  # Concatenate problems and solutions for encoding
        padding=True,
        padding_side="left",
        truncation=True,
        max_length=20, # TODO: Adjust max_length based on model
        return_tensors="pt"
    )

    # mask the labels for the solutions
    labels = encoder["input_ids"].clone()
    for i in range(len(problems)):
        # Find the index of the split token in the input_ids
        split_index = (encoder["input_ids"][i] == split_token_id).nonzero(as_tuple=True)[0]
        # Set the labels to -100 for the problem part, so they won't be used in loss calculation
        labels[i][:(split_index+1)] = -100

    return {
        "input_ids": encoder["input_ids"],
        "attention_mask": encoder["attention_mask"],
        "labels": labels,  # Use the masked labels for loss calculation
    }


# Load the math dataset
def load_math_data(problem_filename="math_problems.txt", solution_filename="math_solutions.txt"):
    import pandas as pd
    problems = [line.strip() for line in open(problem_filename, "r")]
    solutions = [line.strip() for line in open(solution_filename, "r")]
    return pd.DataFrame({"problem": problems, "solution": solutions})

data = load_math_data("math_problems.txt", "math_solutions.txt")

# split into train and test sets
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_data = data[:train_size]
test_data = data[train_size:]

# Initialize tokenizer and dataset
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<|pad|>'})  # Explicitly add a special padding token
    tokenizer.pad_token = '<|pad|>'

train_math_dataset = MathDataset(train_data)
test_math_dataset = MathDataset(test_data)

train_data_loader = DataLoader(train_math_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_data_loader = DataLoader(test_math_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

c:\Users\leonh\anaconda3\envs\adverserialAlignment\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Used to test the Dataloader
# Extract a single batch from the DataLoader
batch = next(iter(train_data_loader))

# Access the batch data
input_ids = batch["input_ids"]
attention_mask = batch["attention_mask"]
labels = batch["labels"]

# Decode and print the batch data
print("Input IDs:", tokenizer.batch_decode(input_ids.tolist(), skip_special_tokens=False))
print("Attention Mask:", attention_mask)
print("Labels:", labels)

Input IDs: ['30 * 51 = 1530<|endoftext|>', '<|pad|>49 + 11 = 60<|endoftext|>', '2726 / 94 = 29<|endoftext|>', '<|pad|>73 * 4 = 292<|endoftext|>', '<|pad|>66 + 35 = 101<|endoftext|>', '<|pad|>65 + 76 = 141<|endoftext|>', '2132 / 52 = 41<|endoftext|>', '93 * 44 = 4092<|endoftext|>', '8 - 89 = -81<|endoftext|>', '21 - 93 = -72<|endoftext|>', '<|pad|>11 * 13 = 143<|endoftext|>', '22 * 28 = 616<|endoftext|>', '<|pad|>82 - 21 = 61<|endoftext|>', '<|pad|>3 + 79 = 82<|endoftext|>', '<|pad|>18 + 41 = 59<|endoftext|>', '<|pad|>59 - 52 = 7<|endoftext|>', '<|pad|>100 - 93 = 7<|endoftext|>', '<|pad|>38 / 1 = 38<|endoftext|>', '<|pad|>94 - 64 = 30<|endoftext|>', '79 * 46 = 3634<|endoftext|>', '1782 / 99 = 18<|endoftext|>', '2240 / 80 = 28<|endoftext|>', '<|pad|>93 + 27 = 120<|endoftext|>', '<|pad|>7 * 72 = 504<|endoftext|>', '45 - 68 = -23<|endoftext|>', '13 - 45 = -32<|endoftext|>', '<|pad|>83 + 17 = 100<|endoftext|>', '62 * 16 = 992<|endoftext|>', '<|pad|>29 + 32 = 61<|endoftext|>', '3726 / 46 = 8

In [3]:
from tqdm.auto import tqdm
from transformers import get_scheduler

num_epochs = 4

# Load the GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

# Define optimizer and training parameters
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = num_epochs * len(train_data_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model.to(device)

# Training loop

for epoch in range(num_epochs):
    progress_bar = tqdm(train_data_loader, desc=f"Epoch {epoch+1} Training Progress") # Add progress bar
    model.train()
    for batch in progress_bar:
        # Move batch to device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        # Update progress bar
        progress_bar.set_postfix(loss=loss.item())

    # validation loss calculation
    model.eval()
    total_val_loss = 0
    total_val_samples = 0
    for batch in test_data_loader:
        with torch.no_grad():
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss = outputs.loss

            batch_size = input_ids.size(0)
            total_val_loss += val_loss.item() * batch_size
            total_val_samples += batch_size
    # Calculate average validation loss
    mean_val_loss = total_val_loss / total_val_samples if total_val_samples > 0 else 0

    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {mean_val_loss:.4f}")
    # Save the finetuned model
    model.save_pretrained("finetuned_gpt2_math_epoch_{}".format(epoch + 1))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
c:\Users\leonh\anaconda3\envs\adverserialAlignment\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Using device: cuda


Epoch 1 Training Progress: 100%|██████████| 1000/1000 [01:13<00:00, 13.64it/s, loss=1.79]


Epoch 1/4, Validation Loss: 1.5242


Epoch 2 Training Progress: 100%|██████████| 1000/1000 [01:12<00:00, 13.76it/s, loss=1.25]


Epoch 2/4, Validation Loss: 1.1724


Epoch 3 Training Progress: 100%|██████████| 1000/1000 [01:11<00:00, 14.08it/s, loss=1.07]


Epoch 3/4, Validation Loss: 1.0280


Epoch 4 Training Progress: 100%|██████████| 1000/1000 [01:11<00:00, 14.02it/s, loss=1.03]


Epoch 4/4, Validation Loss: 0.9710


# Load the Finetuned Model and Run Inference
This section demonstrates how to load the finetuned GPT-2 model and use it to generate solutions for new math problems.

In [4]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load the finetuned model and tokenizer
model_path = "finetuned_gpt2_math_epoch_4"
model = GPT2LMHeadModel.from_pretrained(model_path)

# Ensure the model is on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to generate a solution for a given math problem
def generate_solution(problem, max_length=50):
    model.eval()
    with torch.no_grad():
        # Tokenize the input problem
        input_enc = tokenizer(
            problem,
            return_tensors="pt",
            padding=False,
            truncation=True,
            max_length=max_length
        )
        input_ids = input_enc["input_ids"].to(device)
        attention_mask = input_enc["attention_mask"].to(device)

        # Generate output
        output_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_beams=5,
            early_stopping=True
        )

        # Decode the generated output
        solution = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        return solution

In [5]:
import random
# Select a random example from the test set for inference
random_idx = random.randint(0, len(test_data) - 1)
example_problem = test_data.iloc[random_idx]["problem"]
solution = generate_solution(example_problem)
print(f"Problem: {example_problem}")
print(f"Solution: {solution}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Problem: 89 * 61 =
Solution: 89 * 61 = 5551
